### Install azureml library

In [ ]:
! pip install azureml-core

     |████████████████████████████████| 2.2 MB 5.3 MB/s 
     |████████████████████████████████| 101 kB 9.5 MB/s 
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 547 kB 45.9 MB/s 
     |████████████████████████████████| 40 kB 4.6 MB/s 
     |████████████████████████████████| 147 kB 51.5 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 312 kB 46.8 MB/s 
     |████████████████████████████████| 796 kB 46.7 MB/s 
     |████████████████████████████████| 3.0 MB 22.8 MB/s 
     |████████████████████████████████| 1.3 MB 31.2 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 141 kB 46.5 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 165 kB 47.4 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
     |████████████████████████

### Create workspace from Azure Machine Learning workspace

In [ ]:
from azureml.core import Workspace

source_dir = "./source_dir/"

ws = Workspace.from_config(path = source_dir + "config.json")
print(ws)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EBFWUVXC8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace.create(name='Test-Deployment-Space', subscription_id='9d7c60f0-d3e1-49c3-b70b-080e861838c1', resource_group='Test-Deployment')


### Import libraries

In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json

import tensorflow as tf

### Load tokenizer from saved tokenizer

In [18]:
with open(os.path.join('./outputs/model1/assets/tokenizer.json')) as f:
    data = json.load(f)
    tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)

### Load the maximum sequence length from all the Commonlit provided excerpts stored in txt file

In [20]:
with open('./outputs/model1/assets/max_length.txt', 'r') as text:
    max_length = int(text.read())

### Test loading of pre-trained model

In [21]:
model_load = tf.keras.models.load_model('./outputs/model1')
model_load.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 215, 90)           2581650   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               224256    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 2,806,163
Trainable params: 2,806,163
Non-trainable params: 0
_________________________________________________________________


In [22]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
pred_tokens = tokenizer.texts_to_sequences(pred_text)
pad_pred_tokens = pad_sequences(pred_tokens, maxlen=max_length, padding = 'pre')

### Make predictions and store in np.ndarray

In [23]:
test_preds = model_load.predict(x = pad_pred_tokens)
test_preds

array([[-1.2834516 ],
       [-0.22421896],
       [-0.595436  ],
       [-2.2895405 ],
       [-1.713674  ],
       [-0.2228559 ],
       [-0.02093622]], dtype=float32)

### Convert to list of labels of type 'float'

In [24]:
test_preds_list = test_preds.flatten().tolist()
test_preds_list

[-1.2834515571594238,
 -0.2242189645767212,
 -0.5954359769821167,
 -2.2895405292510986,
 -1.7136739492416382,
 -0.22285589575767517,
 -0.02093621902167797]

### Register saved model on Azure

In [10]:
from azureml.core.model import Model

# Register model
model = Model.register(ws, model_name = "Commonlit-BiLSTM-Seed3", model_path = "./outputs/model1")

NameError: name 'ws' is not defined

### Create environment with necessary dependencies

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

env = Environment(name = "commonlit-test-lstm")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")
conda_dep.add_pip_package("keras")
conda_dep.add_pip_package("pandas")

conda_dep.add_pip_package('tensorflow==2.6.0')
conda_dep.add_pip_package('sklearn')

conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("azureml")
conda_dep.add_pip_package("azureml-contrib-functions")

env.python.conda_dependencies = conda_dep

### Create inference configuration using scoring function

In [ ]:
inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "./echo_score.py",
)

### Create Azure Container Instance and deploy model to container

In [ ]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2)
service = Model.deploy(
    ws,
    "commonlit-bilstm-seed3",
    [model],
    inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-27 02:00:02+00:00 Creating Container Registry if not exists.
2021-08-27 02:00:02+00:00 Registering the environment.
2021-08-27 02:00:05+00:00 Use the existing image.
2021-08-27 02:00:05+00:00 Generating deployment configuration.
2021-08-27 02:00:06+00:00 Submitting deployment to compute..
2021-08-27 02:00:10+00:00 Checking the status of deployment commonlit-bilstm-seed3..
2021-08-27 02:04:53+00:00 Checking the status of inference endpoint commonlit-bilstm-seed3.
Succeeded
ACI service creation operation finished, operation "Succeeded"


### Endpoint Consumption - Test 1

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get("http://16705546-cea5-4e3e-99a7-6274ca5c4289.australiaeast.azurecontainer.io/score")
headers = {"Content-Type": "application/json"}
data = {
    'data': ['Cell division is the process by which a parent cell divides into two or more daughter cells. Cell division usually occurs as part of a larger cell cycle.\n In eukaryotes, there are two distinct types of cell division: a vegetative division, whereby each daughter cell is genetically identical to the parent cell (mitosis), and a reproductive cell division, whereby the number of chromosomes in the daughter cells is reduced by half, to produce haploid gametes (meiosis). \nMeiosis results in four haploid daughter cells by undergoing one round of DNA replication followed by two divisions: homologous chromosomes are separated in the first division, and sister chromatids are separated in the second division.\nBoth of these cell division cycles are used in sexually reproducing organisms at some point in their life cycle, and both are believed to be present in the last eukaryotic common ancestor. Prokaryotes also undergo a vegetative cell division known as binary fission, where their genetic material is segregated equally into two daughter cells. All cell divisions, regardless of organism, are preceded by a single round of DNA replication.']
}
data = json.dumps(data)
response = requests.post(uri, data = data, headers = headers)
print(response.json())

[-2.2895405292510986]


### Endpoint Consumption - Test 2

In [11]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {
    'data': ['Dotty continued to go to Mrs. Gray\'s every night with the milk. Sometimes Katie went with her, and then they always paused a while under the acorn-tree and played "King and Queen." Dotty said she wished they could ever remember to bring their nipperkins, for in that case the milk would taste a great deal more like nectar. The "nipperkins" were a pair of handled cups which the children supposed to be silver, and which they always used at table.\nDotty knew she was doing wrong every time she played "King and Queen." She knew the milk was not hers, but Mrs. Gray\'s; still she said to herself, "Ruthie needn\'t give so much measure, all pressed down and run over. If Queenie and I should drink a great deal more, there would always be a quart left. Yes, I know there would."\nMrs. Gray never said anything about the milk; she merely poured it out in a pan, and gave back the pail to Dotty, asking her at the same time as many questions as the child would stay to hear.',
             'Cell division is the process by which a parent cell divides into two or more daughter cells. Cell division usually occurs as part of a larger cell cycle.\n In eukaryotes, there are two distinct types of cell division: a vegetative division, whereby each daughter cell is genetically identical to the parent cell (mitosis), and a reproductive cell division, whereby the number of chromosomes in the daughter cells is reduced by half, to produce haploid gametes (meiosis). \nMeiosis results in four haploid daughter cells by undergoing one round of DNA replication followed by two divisions: homologous chromosomes are separated in the first division, and sister chromatids are separated in the second division.\nBoth of these cell division cycles are used in sexually reproducing organisms at some point in their life cycle, and both are believed to be present in the last eukaryotic common ancestor. Prokaryotes also undergo a vegetative cell division known as binary fission, where their genetic material is segregated equally into two daughter cells. All cell divisions, regardless of organism, are preceded by a single round of DNA replication.'],
}

body = str.encode(json.dumps(data))

url = 'http://16705546-cea5-4e3e-99a7-6274ca5c4289.australiaeast.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))


URLError: <urlopen error [Errno 11001] getaddrinfo failed>